In [1]:
import pandas as pd
from glob import glob
import numpy as np

In [2]:
df= pd.read_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/raw_data/N4_filtered-annotated-peaks_minfoldch5_minus-2000bp-to-plus-500bp_111623.tsv', sep='\t', header=0)
df = df.loc[df['species']=="Arabidopsis_thaliana_Col-0"]

In [ ]:
#reading TF label df
labels = pd.read_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/raw_data/ath-258-tf-info_simple.csv', sep=',', header=0)

# counting number of members of each TF family
fam_count = labels['tf_family'].value_counts().reset_index(drop=False)
fam_count= pd.DataFrame(fam_count).rename(columns={"index": "tf_family", "count": "fam_count"})

labels_count = labels.merge(fam_count, on = 'tf_family')
labels_count.head()

,gene_id,tf_family,tf_name,tf_and_fam,representative_category,tf_clade_curated,clade,fam_count
0,AT5G60130,ABI3VP1,AT5G60130,ABI3VP1_AT5G60130,1,2,ABI3VP1_c2,3
1,AT4G01500,ABI3VP1,NGA4,ABI3VP1_NGA4,0,1,ABI3VP1_c1,3
2,AT3G18990,ABI3VP1,VRN1,ABI3VP1_VRN1,2,3,ABI3VP1_c3,3
3,AT5G64750,AP2EREBP,ABR1,AP2EREBP_ABR1,2,1,AP2EREBP_c1,43
4,AT1G01250,AP2EREBP,AT1G01250,AP2EREBP_AT1G01250,0,1,AP2EREBP_c1,43


In [4]:
# adding tf info to large table 
dfl = df.merge(labels_count, left_on = 'tf', right_on = "gene_id", how = "left")
dfl.head()

,peak_chr,peak_start,peak_end,peak_name,peak_foldch,peak_pscore,peak_qscore,peak_summit,feature_chr,feature_source,...,n_species_in_orthogroup,n_cons_species_clade_aware,gene_id,tf_family,tf_name,tf_and_fam,representative_category,tf_clade_curated,clade,fam_count
0,Chr3,15645684,15646225,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,121.602,1156.46,1152.01,268,Chr3,phytozomev12,...,4,3,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
1,Chr1,1637949,1638542,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,115.549,1085.10,1080.68,228,Chr1,phytozomev12,...,4,1,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
2,Chr1,1637949,1638542,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,115.549,1085.10,1080.68,228,Chr1,phytozomev12,...,4,1,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
3,Chr1,18204823,18205351,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,115.549,1085.10,1080.68,268,Chr1,phytozomev12,...,4,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
4,Chr1,17178696,17179222,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,110.414,1025.15,1020.76,268,Chr1,phytozomev12,...,4,1,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21


In [5]:
# creating bins to classify summit_to_transcript_end
l1 = list(range(-2000, 500, 50))
l2 = list(range(-1950, 550, 50))

bins_df = pd.DataFrame(zip(l1, l2), columns = ['start', 'end'])

bins_df['mid'] = (bins_df['start'] + bins_df['end'])/2

bins_df.head()

,start,end,mid
0,-2000,-1950,-1975.0
1,-1950,-1900,-1925.0
2,-1900,-1850,-1875.0
3,-1850,-1800,-1825.0
4,-1800,-1750,-1775.0


In [6]:
def dist_bins(summit_to_cds_start):
    for index, row in bins_df.iterrows():
        if (summit_to_cds_start >= row['start']) and (summit_to_cds_start < row['end']):
            return(row['mid'])
       # if (summit_to_cds_end < -5000):
       #     return(-4975.0)

# running function over every row
dfl['s2cds_bin'] = dfl['summit_to_cds_start'].apply(lambda x: dist_bins(x))
display(dfl)

,peak_chr,peak_start,peak_end,peak_name,peak_foldch,peak_pscore,peak_qscore,peak_summit,feature_chr,feature_source,...,n_cons_species_clade_aware,gene_id,tf_family,tf_name,tf_and_fam,representative_category,tf_clade_curated,clade,fam_count,s2cds_bin
0,Chr3,15645684,15646225,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,121.60200,1156.4600,1152.01000,268,Chr3,phytozomev12,...,3,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-525.0
1,Chr1,1637949,1638542,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,115.54900,1085.1000,1080.68000,228,Chr1,phytozomev12,...,1,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-1175.0
2,Chr1,1637949,1638542,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,115.54900,1085.1000,1080.68000,228,Chr1,phytozomev12,...,1,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-325.0
3,Chr1,18204823,18205351,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,115.54900,1085.1000,1080.68000,268,Chr1,phytozomev12,...,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-875.0
4,Chr1,17178696,17179222,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,110.41400,1025.1500,1020.76000,268,Chr1,phytozomev12,...,1,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378470,Chr4,14684381,14684529,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.01329,10.1066,7.88126,77,Chr4,phytozomev12,...,1,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-975.0
1378471,Chr4,1400681,1400817,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.01329,10.1066,7.88126,69,Chr4,phytozomev12,...,1,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-75.0
1378472,Chr4,14684381,14684529,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.01329,10.1066,7.88126,77,Chr4,phytozomev12,...,1,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-1125.0
1378473,Chr1,28683405,28683550,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.01329,10.1066,7.88126,70,Chr1,phytozomev12,...,2,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-475.0


In [7]:
cds_bin_count = dfl.groupby(["tf", 's2cds_bin', 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index()
cds_bin_count = pd.DataFrame(cds_bin_count).rename(columns={"peak_name": "s2cds_peak_bin_count"})

In [8]:
cds_bin_count = dfl.groupby(["tf", 's2cds_bin', 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index()
cds_bin_count = pd.DataFrame(cds_bin_count).rename(columns={"peak_name": "s2cds_peak_bin_count"})

peak_count = dfl.groupby(["tf", 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index().rename(columns={"peak_name": "tf_by_cons_peak_count"})
peak_count_fam = dfl.groupby(["tf"])['peak_name'].agg('count').reset_index().rename(columns={"peak_name": "alltf_peak_count"})

display(peak_count_fam)

display(peak_count)


,tf,alltf_peak_count
0,AT1G01060,7021
1,AT1G01250,2022
2,AT1G01720,2657
3,AT1G02230,7133
4,AT1G02250,9534
...,...,...
239,AT5G66300,5320
240,AT5G66700,9987
241,AT5G66940,12816
242,AT5G67300,3393


,tf,n_cons_species_minfrac0,tf_by_cons_peak_count
0,AT1G01060,1,1483
1,AT1G01060,2,2238
2,AT1G01060,3,2123
3,AT1G01060,4,1177
4,AT1G01250,1,568
...,...,...,...
970,AT5G67300,4,414
971,AT5G67580,1,784
972,AT5G67580,2,628
973,AT5G67580,3,448


In [ ]:
peak_count = peak_count.merge(peak_count_fam, on= ['tf'], how = 'left')
bin_count = cds_bin_count.merge(peak_count, on = ['tf', 'n_cons_species_minfrac0'], how = "left")

bin_count

,tf,s2cds_bin,n_cons_species_minfrac0,s2cds_peak_bin_count,tf_by_cons_peak_count,alltf_peak_count
0,AT1G01060,-1975.0,1,15,1483,7021
1,AT1G01060,-1975.0,2,32,2238,7021
2,AT1G01060,-1975.0,3,41,2123,7021
3,AT1G01060,-1975.0,4,15,1177,7021
4,AT1G01060,-1925.0,1,26,1483,7021
...,...,...,...,...,...,...
46737,AT5G67580,425.0,4,4,209,2069
46738,AT5G67580,475.0,1,20,784,2069
46739,AT5G67580,475.0,2,13,628,2069
46740,AT5G67580,475.0,3,7,448,2069


In [ ]:
bin_count['s2cds_prop'] = bin_count['s2cds_peak_bin_count'] / bin_count['tf_by_cons_peak_count'] 
bin_count['tf_pcount'] = bin_count['tf'] + "_" +  bin_count['alltf_peak_count'].astype(str)  
display(bin_count)

,tf,s2cds_bin,n_cons_species_minfrac0,s2cds_peak_bin_count,tf_by_cons_peak_count,alltf_peak_count,s2cds_prop,tf_pcount
0,AT1G01060,-1975.0,1,15,1483,7021,0.010115,AT1G01060_7021
1,AT1G01060,-1975.0,2,32,2238,7021,0.014298,AT1G01060_7021
2,AT1G01060,-1975.0,3,41,2123,7021,0.019312,AT1G01060_7021
3,AT1G01060,-1975.0,4,15,1177,7021,0.012744,AT1G01060_7021
4,AT1G01060,-1925.0,1,26,1483,7021,0.017532,AT1G01060_7021
...,...,...,...,...,...,...,...,...
46737,AT5G67580,425.0,4,4,209,2069,0.019139,AT5G67580_2069
46738,AT5G67580,475.0,1,20,784,2069,0.025510,AT5G67580_2069
46739,AT5G67580,475.0,2,13,628,2069,0.020701,AT5G67580_2069
46740,AT5G67580,475.0,3,7,448,2069,0.015625,AT5G67580_2069


In [11]:
len(bin_count['tf'].unique())

244

In [12]:
# saving table 
bin_count.to_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/analysis/peak_distribution/distance_start_bin_prop_by_tf_cons.tsv' , sep='\t', index=False)
